# Classifiers

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.factset import EquityMetadata

start_date = '2014-01-02'
end_date = '2019-10-31' 

## Grouping and Filtering by Exchanges

In the example below, you will see an example of how classifiers can be used to group and filter a pipeline by a particular listing exchange.

[EquityMetadata](https://www.quantopian.com/docs/api-reference/pipeline-api-reference#quantopian.pipeline.data.factset.EquityMetadata) object holds various information about an equity's metadata. 

In [2]:
# Since the underlying data of EquityMetadata.listing_exchange
# is of type object (string), .latest returns a classifier.
exchange = EquityMetadata.listing_exchange.latest

e_pipe = Pipeline(
    columns={
        'exchange': exchange
    },
    screen=(exchange.eq('NYS'))
)

e_mdf = run_pipeline(e_pipe, start_date, start_date)
e_mdf['exchange'].value_counts()

NYS     3101
OTC        0
PSE        0
BATS       0
NAS        0
CBO        0
ASE        0
CHI        0
Name: exchange, dtype: int64

## Group PE Ratio into Deciles

We will learn more about this in section 4, but this is the operation classifier is often used for. Deciles are the division of values into 10 bins. In the code below, we group the PE ratios of all assets into four groups. We also remove all assets with `NaN` PE Ratio by creating a filter using `notnan()` function. 

In [4]:
from quantopian.pipeline.data.morningstar import Fundamentals

def make_pipeline():
    return Pipeline(
        columns={
            'pe_ratio': Fundamentals.pe_ratio.latest,
            'pe_decile': Fundamentals.pe_ratio.latest.deciles()
        },
        screen=Fundamentals.pe_ratio.latest.notnan()
    )

pe_pipe = make_pipeline()
pe_mdf = run_pipeline(pe_pipe, start_date, start_date)
pe_mdf['pe_decile'].value_counts()

6    456
0    456
7    455
4    455
2    454
1    454
9    453
3    451
5    451
8    450
Name: pe_decile, dtype: int64

In [5]:
pe_mdf.head(5)

pe_decile  pe_ratio
2014-01-02 00:00:00+00:00 Equity(2 [ARNC])           7   35.2113
                          Equity(21 [AAME])          0    8.8731
                          Equity(24 [AAPL])          2   13.9470
                          Equity(31 [ABAX])          7   52.6316
                          Equity(39 [DDC])           8   62.8931

## Conclusion

By grouping the values into deciles, you can already tap into interesting analyses. For instance, you may simply get deciles of several factors and combine them to calculate stock ranks. High-ranked stocks are then assigned long positions while shorting low-ranked ones.